# Price Agent

The ``PriceThresholdSwapAgent`` is a reactive DeFi Agent that continuously monitors token prices in a specified liquidity pool, such as a Uniswap V2 pair for USDC/WETH. It automatically triggers a predefined swap (e.g., selling WETH for USDC) when the price exceeds a user-set threshold, integrating real-time event feeds for efficient execution

* To download notebook to this tutorial, see [here](https://github.com/defipy-devs/defipy-docs/blob/main/docs/onchain/onchain/price_threshold_swap.ipynb) 

### 📘 Notable Classes

---

* **Class**: 📘 ``defipy.agents.config.PriceThresholdConfig`` 
    * **Purpose**: Configuration for price threshold swap agent.
        * **Parameters**:
            * `threshold`: Price threshold to check (`float`).
            * `pool_address`: UniV2 pool address (`str`).
            * `provider_url`: Provider URL (eg, infura.io)(`str`).
            * `platform`: Platform where pool resides (eg, uniswap) (`str`).
            * `abi_name`: ABI handle pointing to pool contract in JSON (eg, UniswapV2Pair) (`str`).

---
         
* **Class**: 📘 ``defipy.agents.PriceThresholdSwapAgent`` 
    * **Purpose**: Reactive DeFi agent for determining price threshold.
        * **Parameters**:
            * `config`: Agent configuration parameters (`PriceThresholdConfig`).
    * **Methods**:
        * ``apply()``
            * **Purpose**: Apply price threshold agent   
        * ``get_token_price(tkn1_over_tkn0: bool, block_num: int)``
            * **Purpose**: Get current price of pool  
            * **Parameters**:
                * `tkn1_over_tkn0`: price = tkn1/tkn0 (true) (`bool`).
                * `block_num`: block number (`int`).
        * ``check_condition(threshold: float, tkn1_over_tkn0: bool)``
            * **Purpose**: Calculate initial token value of position   
            * **Parameters**:
                * `threshold`: override config price threshold (optional) (`float`).
                * `tkn1_over_tkn0`: price = tkn1/tkn0 (true, optional) (`float`).
        * ``get_w3()``
            * **Purpose**: Getter function for web3.py connector object
        * ``get_abi()``
            * **Purpose**: Getter function for ABI json data
        * ``get_contract_instance()``
            * **Purpose**: Getter function for pool contract instance
        * ``get_lp_data()``
            * **Purpose**: Getter function for pool data associated with `pool_address` from config


In [1]:
from defipy import *
from web3scout import *

### Setup Agent

In [2]:
price_threshold = 3000.0
swap_amount = 1.0
pair_address = "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc"
provider_url = "https://mainnet.infura.io/v3/9624e3e5c40f4ac3958b79fa5aa2562d"
platform = Platform.AGNOSTIC
abi_name = JSONContract.UniswapV2Pair

config = PriceThresholdConfig(
    threshold = price_threshold,
    swap_amount = swap_amount,
    pool_address = pair_address,
    provider_url = provider_url,
    platform = platform,
    abi_name = abi_name,
)

agent = PriceThresholdSwapAgent(config)

print(f"Monitoring price movements @ pool address {pair_address}")

Monitoring price movements @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc


### Agent Run #1

In [3]:
# Apply agent
agent.apply()

# Grab agent data
tkn1_over_tkn0 = True
price = agent.get_token_price(True)
price_condition_pass = agent.check_condition()
contract_instance = agent.get_contract_instance()
lp_data = agent.get_lp_data()

tkn0 = lp_data.tkn0; tkn1 = lp_data.tkn1; reserves = lp_data.reserves

# Print agent data
print("---------------------------------------------------------------------------------------")
print(f"Agent data @ pool address {pair_address}")
print("---------------------------------------------------------------------------------------")
print(f"reserve0 = {reserves[0]/(10**tkn0.token_decimal):.2f} {tkn0.token_name} @ token address {tkn0.token_addr}")
print(f"reserve1 = {reserves[1]/(10**tkn1.token_decimal):.2f} {tkn1.token_name} @ token address {tkn1.token_addr}")

if(tkn1_over_tkn0):
    price = (reserves[0] / reserves[1]) * (10 ** (tkn1.token_decimal - tkn0.token_decimal))
    print(f"\n{tkn1.token_name} Price in {tkn0.token_name}: {price}")
    print(f"Threshold PASS, {tkn1.token_name} Price > {price_threshold}: {price_condition_pass}")
else:
    price = (reserves[1] / reserves[0]) * (10 ** (tkn0.token_decimal - tkn1.token_decimal))
    print(f"\n{tkn0.token_name} Price in {tkn1.token_name}: {price}")
    print(f"Threshold PASS, {tkn0.token_name} Price > {price_threshold}: {price_condition_pass}")

---------------------------------------------------------------------------------------
Agent data @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc
---------------------------------------------------------------------------------------
reserve0 = 14902246.41 USDC @ token address 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
reserve1 = 3867.84 WETH @ token address 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2

WETH Price in USDC: 3852.855206255951
Threshold PASS, WETH Price > 3000.0: True


### Agent Run #2

In [4]:
price_threshold = 0.0003
tkn1_over_tkn0 = False

# Apply agent
agent.apply()

# Grab agent data
price = agent.get_token_price(True)
price_condition_pass = agent.check_condition(price_threshold, tkn1_over_tkn0)
contract_instance = agent.get_contract_instance()
lp_data = agent.get_lp_data()

tkn0 = lp_data.tkn0; tkn1 = lp_data.tkn1; reserves = lp_data.reserves

# Print agent data
print("---------------------------------------------------------------------------------------")
print(f"Agent data @ pool address {pair_address}")
print("---------------------------------------------------------------------------------------")
print(f"reserve0 = {reserves[0]/(10**tkn0.token_decimal):.2f} {tkn0.token_name} @ token address {tkn0.token_addr}")
print(f"reserve1 = {reserves[1]/(10**tkn1.token_decimal):.2f} {tkn1.token_name} @ token address {tkn1.token_addr}")

if(tkn1_over_tkn0):
    price = (reserves[0] / reserves[1]) * (10 ** (tkn1.token_decimal - tkn0.token_decimal))
    print(f"\n{tkn1.token_name} Price in {tkn0.token_name}: {price}")
    print(f"Threshold PASS, {tkn1.token_name} Price > {price_threshold}: {price_condition_pass}")
else:
    price = (reserves[1] / reserves[0]) * (10 ** (tkn0.token_decimal - tkn1.token_decimal))
    print(f"\n{tkn0.token_name} Price in {tkn1.token_name}: {price}")
    print(f"Threshold PASS, {tkn0.token_name} Price > {price_threshold}: {price_condition_pass}")

---------------------------------------------------------------------------------------
Agent data @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc
---------------------------------------------------------------------------------------
reserve0 = 14902246.41 USDC @ token address 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
reserve1 = 3867.84 WETH @ token address 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2

USDC Price in WETH: 0.00025954777599123936
Threshold PASS, USDC Price > 0.0003: False


### Agent Run #3

In [5]:
abi = ABILoad(platform, abi_name)
connect = ConnectW3(provider_url)
connect.apply()

In [6]:
last_block = connect.get_w3().eth.block_number
start_block = last_block - 100

# Sync
rEvents = RetrieveEvents(connect, abi)
result_events = rEvents.apply(EventType.SYNC, address = pair_address, start_block=start_block, end_block=last_block)
rEvents.to_dataframe(result_events).head(2)

,blockNumber,event,address,blockHash,logIndex,transactionHash,transactionIndex,args
0,23036229,Sync,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0x0ac80d5e85cb5afd7af35b00abe01b0fb13956d31a54...,168,0xf159205619755096b29e593a77b1f953ced286e37c36...,19,"{'reserve0': 14903902876206, 'reserve1': 38674..."
1,23036233,Sync,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0x0809dac0bef189764149021e9986b21d75d929f19541...,237,0x941c7d14bc11a88505a8af64130ac7f4159399e02e9b...,50,"{'reserve0': 14903909876206, 'reserve1': 38674..."


In [7]:
lp = agent.prime_pool_state(start_block, 'user')
lp.summary() 

Exchange USDC-WETH (LP)
Reserves: USDC = 14903625.326206, WETH = 3867.4831854328822
Liquidity: 0.08984440448421524 



In [8]:
agent.run(result_events, lp, tkn0)

Block 23036229: Swapped 1.0 USDC for 0.000258720975167816 WETH
Block 23036233: Swapped 1.0 USDC for 0.000258720940500699 WETH
Block 23036233: Swapped 1.0 USDC for 0.00025872090583359 WETH
Block 23036247: Swapped 1.0 USDC for 0.000258720871166487 WETH
Block 23036251: Swapped 1.0 USDC for 0.000258720836499391 WETH
Block 23036253: Swapped 1.0 USDC for 0.000258720801832302 WETH
Block 23036266: Swapped 1.0 USDC for 0.000258720767165221 WETH
Block 23036272: Swapped 1.0 USDC for 0.000258720732498146 WETH
Block 23036277: Swapped 1.0 USDC for 0.000258720697831078 WETH
Block 23036279: Swapped 1.0 USDC for 0.000258720663164017 WETH
Block 23036287: Swapped 1.0 USDC for 0.000258720628496963 WETH
Block 23036291: Swapped 1.0 USDC for 0.000258720593829916 WETH
Block 23036292: Swapped 1.0 USDC for 0.000258720559162876 WETH
Block 23036303: Swapped 1.0 USDC for 0.000258720524495843 WETH
Block 23036309: Swapped 1.0 USDC for 0.000258720489828817 WETH


In [11]:
price = agent.get_token_price(True, last_block)
price

3852.855206255951